In [ ]:
**Intro**

I decided to take a stab at this project because it looked like a good excercise in unstructured data. Also, the subject really interested me. 

The initial approach I took with this question was analyzing word counts over time, specifically around the time of the crash. The collapse of tulip bulb prices occured in Febuary of 1637, therefore I decided to look at the time period of 1634-1637. This period probably should be extended, but I kept it small to save on time since this is only an exercise.

The data set is a collection of Dutch newspapers from 1618 to 1699. It was scraped from non-electronic copies using optical character recognition, therefore there are quite a few errors as one could imagine translating 17th century Dutch text into a modern day alphabet.

Let's read in some packages and the CSV file.

In [ ]:
library(ggplot2, warn.conflicts = FALSE)
library(reshape2, warn.conflicts = FALSE)
library(plyr, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)
library(tidytext, warn.conflicts = FALSE)
library(broom, warn.conflicts = FALSE)


#Read in CSV
dutch_newspapers.df <- read.csv("../input/dutch_newspapers.csv")
dutch_newspapers.df[1,]

I made some changes to the dataframe right off the back. That included adding a "0" to all the single digit "month" variables. I removed all punctuation in "text" and replaced them with "", the case could just as easily be made to replace punctuation with " ", but for now let's start with "".  Then, I created a "date" variable that combines the "month" and "year", as a numeric with the year coming first. I did this because I decided on using months as my smallest date increment. If I want to order by date, I can very easily order by this "date" variable. 

In [ ]:
dutch_newspapers.df <- transform(dutch_newspapers.df,
                                 month = ifelse((month < 10), paste(0, month, sep = ""), month), 
                                 text = gsub("[[:punct:]]", "", as.character(text)),
                                 date = as.numeric(paste(year, ifelse((month < 10), paste(0, month, sep = ""), month), sep = ".")))  

Using the extremely useful unnest_tokens function in the package TidyText, I selected from the "text" columns every word individually. I then removed words that were less than one character in length, I figured that would eliminate a large portion of the dataframe and single letter words are not extremely useful in this analysis.

In [ ]:
dutch_newspapers.df <- unnest_tokens(dutch_newspapers.df[dutch_newspapers.df$date <= 1638.01 & dutch_newspapers.df$date >= 1634.01,], word, text)

dutch_newspapers.df <- dutch_newspapers.df[nchar(dutch_newspapers.df$word) > 2,]
head(dutch_newspapers.df)

Next, let's reshape the dataframe so that each word has its own column. Then, remove all the words that only appear once. For each month create a column that counts the total amount of words written that month, we will use this to normalize the appearance of words from month to month, since some months may have many more words written than others.  Next, each count was divided by the total number of words counted that month. 

In [ ]:
#Count the words, dcast the dataframe
word.count <- dcast(count(dutch_newspapers.df, date, word), date ~ word)

#Remove words that only have 1 character
word.count <- word.count[,colSums(word.count, na.rm = TRUE) >=2]

#Sum the rows, creatin a total words used for the month
word.count$sum <- rowSums(word.count[, 2:8459], na.rm = TRUE)

word.count.later <- word.count
word.count.later[is.na(word.count.later)] <- 0

#Create ratios to normalize the counts
word.count[, -c(1,8460)] <- word.count[, -c(1,8460)]/word.count[, c(8460)]

head(word.count[c(1:5,8460)])

In order to tie the word counts in with the actual tulip bulb market crash, I decided to use the change in bulb prices and correlate the counts of each words with the bulb prices.  There is not very much information on tulip bulb prices around the time of the crash, the prices I could find I copied into a dataframe. 

In [ ]:
datenum <- c( 6, 22, 34, 36, 37, 38)
date <- c( 1634.06, 1635.10, 1636.10, 1636.12, 1637.01, 1637.02)
price <- c( 1, 2, 4, 10, 15, 60)

price.df <- data.frame(datenum, date, price)
 
plot(datenum, price)

Next, I created two models to infer the missing price points. One is an exponential model fitted to two of the researched data points, the other is a LOESS line fitted to all the researched data points.

In [ ]:
plot(datenum, price)
#Two point exponential curve prediction

#Exponential curve created using two points (6,1), (38,60). 1 = a*b^6 and 60 = a*b^38. Solve for a and b.
# 60/1 = (a*b^38)/(a*b^6) -> 60 = b^32 -> b = 60^(1/32)
# 1 = a*(60^(1/32))^6 -> a = 1/((60^(1/32))^6)
b <- 60^(1/32)
a <- 1/((60^(1/32))^6)

lines(datenum, a*(b^datenum), col = "red")

two.point.function <- function(date){
  b = 60^(1/32)
  a = 1/((60^(1/32))^6)
  price.var = a*(b^date)
  return(price.var)
  }

b = 60^(1/32)
a = 1/((60^(1/32))^6)

#Loess prediction

loess.model <- suppressWarnings(loess(price ~ datenum, span = .4))
lines(datenum, predict(loess.model), col = "blue")

Created a dataframe with the estimated price points for all the dates I am analyzing. I set all the post-crash price points to 0; even though this may not be accurate, there was no information I could find on post-crash prices.

In [ ]:
v.date <- c()
for(i in 1634:1637){
  for(j in 1:12){
    g = as.numeric(paste( i, ifelse((j < 10), paste(0, j, sep = ""), j), sep = "."))
    v.date <- append(v.date, g)
  }
}

price.df <- as.data.frame(v.date)
price.df$datenum <- as.numeric(row.names(price.df))


price.df <- transform(price.df, two.point = ifelse(datenum <= 38, two.point.function(datenum), 0),
                                loess = ifelse(datenum <= 5, two.point.function(datenum), ifelse(datenum <= 38, predict(loess.model, datenum), 0)))
head(price.df)

Bind that dataframe with the word.count dataframe.

In [ ]:
count.model.df <- cbind(word.count, price.df[(price.df$v.date %in% word.count$date),])

#Reorder
count.model.df <- count.model.df[c(1,8460,8462:8464,2:8459)]

#Change NA's to 0
count.model.df[is.na(count.model.df)] <- 0

head(count.model.df[c(1:8)])

Let's run correlation tests for each word with the "two point" model price points. Words returned are those with p-values less than 0.05.

Let's take a look at the top four highest correlated words, with at least 15 occurences.

In [ ]:
pvalue <- c()
index <- c()
word <- c()
cor <- c()
wordcount <- c()

for(i in 6:8463){
  test <- cor.test(count.model.df[,4], count.model.df[,i])
  if(test$p.value <= 0.05){
    pvalue = append(pvalue, test$p.value)
    index = append(index, i)
    word = append(word, colnames(count.model.df[i]))
    cor = append(cor, test$estimate)
    wordcount = append(wordcount, sum(word.count.later[[colnames(count.model.df[i])]]))
  }
}

two.point.df <- data.frame(index, word, pvalue, cor, wordcount)[order(abs(cor), decreasing = TRUE),]
head(two.point.df[two.point.df$wordcount >= 15 & abs(two.point.df$cor) >= .55,], n = 4)

Let's see how they look visually.

In [ ]:
par(mar = c(5,5,2,5),mfrow = c(2, 2))
#ften
with(count.model.df, plot(datenum, ften, col="red3",pch=16, 
                            ylab=expression('"ften"')))
par(new = T)
with(count.model.df, plot(datenum, two.point, type = "l", lwd = 2, axes=F, xlab=NA, ylab=NA, cex=1.2))
axis(side = 4)
mtext(side = 4, line = 3, 'Tulip Bulp Price: TwoPoint')

#opapost
with(count.model.df, plot(datenum, opapost, col="blue",pch=16, 
                            ylab=expression('"opapost"')))
par(new = T)
with(count.model.df, plot(datenum, two.point, type = "l", lwd = 2, axes=F, xlab=NA, ylab=NA, cex=1.2))
axis(side = 4)
mtext(side = 4, line = 3, 'Tulip Bulp Price: TwoPoint')

#epnbe
with(count.model.df, plot(datenum, epnbe, col="orange", pch=16, 
                          ylab=expression('"epnbe"')))
par(new = T)
with(count.model.df, plot(datenum, two.point, type = "l", lwd = 2, axes=F, xlab=NA, ylab=NA, cex=1.2))
axis(side = 4)
mtext(side = 4, line = 3, 'Tulip Bulp Price: TwoPoint')

#fgne
with(count.model.df, plot(datenum, fgne, col="green", pch=16, 
                          ylab=expression('"fgne"')))
par(new = T)
with(count.model.df, plot(datenum, two.point, type = "l", lwd = 2, axes=F, xlab=NA, ylab=NA, cex=1.2))
axis(side = 4)
mtext(side = 4, line = 3, 'Tulip Bulp Price: TwoPoint')


Repeat with the LOESS model.

In [ ]:
pvalue <- c()
index <- c()
word <- c()
cor <- c()
wordcount <- c()

for(i in 6:8463){
  test <- cor.test(count.model.df[,5], count.model.df[,i])
  if(test$p.value <= 0.05){
    pvalue = append(pvalue, test$p.value)
    index = append(index, i)
    word = append(word, colnames(count.model.df[i]))
    cor = append(cor, test$estimate)
    wordcount = append(wordcount, sum(word.count.later[[colnames(count.model.df[i])]]))
  }
}

loess.df <- data.frame(index, word, pvalue, cor, wordcount)[order(abs(cor), decreasing = TRUE),]
head(loess.df[loess.df$wordcount >= 15 & abs(loess.df$cor) >= .55,], n = 4)

Here I decided to use a lag function, to see if there were any words that mirrored the price of tulip bulbs but one month in advance, this is a word that could be used as a predictor. First the "two point" with lag.

In [ ]:
par(mar = c(5,5,2,5),mfrow = c(2, 2))

with(count.model.df, plot(datenum, opapost, col="red3",pch=16, 
                          ylab=expression('"opapost"')))

par(new = T)
with(count.model.df, plot(datenum, loess, type = "l", lwd = 2, axes=F, xlab=NA, ylab=NA, cex=1.2))
axis(side = 4)
mtext(side = 4, line = 3, 'Tulip Bulp Price: Loess')



with(count.model.df, plot(datenum, ften, col="blue",pch=16, 
                          ylab=expression('"ften"')))

par(new = T)
with(count.model.df, plot(datenum, loess, type = "l", lwd = 2, axes=F, xlab=NA, ylab=NA, cex=1.2))
axis(side = 4)
mtext(side = 4, line = 3, 'Tulip Bulp Price: Loess')



with(count.model.df, plot(datenum, croupen, col="orange", pch=16, 
                          ylab=expression('"croupen"')))

par(new = T)
with(count.model.df, plot(datenum, loess, type = "l", lwd = 2, axes=F, xlab=NA, ylab=NA, cex=1.2))
axis(side = 4)
mtext(side = 4, line = 3, 'Tulip Bulp Price: Loess')


with(count.model.df, plot(datenum, gun, col="green", pch=16, 
                          ylab=expression('"gun"')))

par(new = T)
with(count.model.df, plot(datenum, loess, type = "l", lwd = 2, axes=F, xlab=NA, ylab=NA, cex=1.2))
axis(side = 4)
mtext(side = 4, line = 3, 'Tulip Bulp Price: Loess')

**Summary**

Further study into the occurences of the words discovered from this analysis would be needed for any significant conclusions. An interesting observation is that the word "croupen" is an old word used to describe a respiratory infection. In the  research I did about the market collapse, there were repeated references to an outbreak of the plague that may have lead to the crash. The increasing occurence of the word "croupen" may correlate with the increasing worry of the plague outbreak. 

Other approaches to this problem would be to use time series analysis of word occurences over many years and see if any words have seasonal trends. If so, did those trends change dramatically around the time of the crash. Another approach is using sentiment analysis to analyze trends in word sentiments. This would require a database of 17th century dutch words with their sentiments attached.